In [ ]:
from os.path import join as pjoin
from common.skeleton import Skeleton
import numpy as np
import os
from common.quaternion import *
from paramUtil import *
import torch
from tqdm import tqdm
import os
import pandas as pd
import re
from scipy.spatial.transform import Rotation
import pickle
import math
from tqdm import tqdm


In [ ]:
def uniform_skeleton(positions, target_offset):
    src_skel = Skeleton(n_raw_offsets, animo_kinematic_chain, 'cpu')
    src_offset = src_skel.get_offsets_joints(torch.from_numpy(positions[0]))
    src_offset = src_offset.numpy()
    tgt_offset = target_offset.numpy()
    '''Calculate Scale Ratio as the ratio of legs'''
    src_leg_len = np.abs(src_offset[l_idx1]).max() + np.abs(src_offset[l_idx2]).max()
    tgt_leg_len = np.abs(tgt_offset[l_idx1]).max() + np.abs(tgt_offset[l_idx2]).max()

    scale_rt = tgt_leg_len / src_leg_len
    # print(scale_rt)
    src_root_pos = positions[:, 0]
    tgt_root_pos = src_root_pos * scale_rt

    '''Inverse Kinematics'''
    quat_params = src_skel.inverse_kinematics_np(positions, face_joint_indx)
    # print(quat_params.shape)

    '''Forward Kinematics'''
    src_skel.set_offset(target_offset)
    new_joints = src_skel.forward_kinematics_np(quat_params, tgt_root_pos)
    return new_joints


In [ ]:
def process_file(positions, feet_thre):
    positions = uniform_skeleton(positions, tgt_offsets)
    '''Put on Floor'''
    floor_height = positions.min(axis=0).min(axis=0)[1]
    positions[:, :, 1] -= floor_height

    '''XZ at origin'''
    root_pos_init = positions[0]
    root_pose_init_xz = root_pos_init[0] * np.array([1, 0, 1])
    positions = positions - root_pose_init_xz

    def rotate_pos(positions):
        """
        Rotate all positions 180 degrees around the Z-axis.
        
        :param positions: numpy array of shape (n_frames, n_joints, 3)
        :return: rotated positions
        """
        r = Rotation.from_rotvec(np.array([-np.pi/2, 0, 0])) 
        
        # Apply rotation to all positions
        rotated_positions = r.apply(positions.reshape(-1, 3)).reshape(positions.shape)
        r = Rotation.from_rotvec(np.array([0, -np.pi/2, 0]))  # pi radians = 180 degrees
        rotated_positions = r.apply(rotated_positions.reshape(-1, 3)).reshape(positions.shape)
        
        
        return rotated_positions

    positions = rotate_pos(positions)

    '''Put on Floor Again'''
    floor_height = positions.min(axis=0).min(axis=0)[1]
    positions[:, :, 1] -= floor_height
    '''New ground truth positions'''
    global_positions = positions.copy()
    """ Get Foot Contacts """

    def foot_detect(positions, thres):
        velfactor, heightfactor = np.array([thres, thres]), np.array([3.0, 2.0])

        feet_l_x = (positions[1:, fid_l, 0] - positions[:-1, fid_l, 0]) ** 2
        feet_l_y = (positions[1:, fid_l, 1] - positions[:-1, fid_l, 1]) ** 2
        feet_l_z = (positions[1:, fid_l, 2] - positions[:-1, fid_l, 2]) ** 2
        feet_l = ((feet_l_x + feet_l_y + feet_l_z) < velfactor).astype(np.float32)

        feet_r_x = (positions[1:, fid_r, 0] - positions[:-1, fid_r, 0]) ** 2
        feet_r_y = (positions[1:, fid_r, 1] - positions[:-1, fid_r, 1]) ** 2
        feet_r_z = (positions[1:, fid_r, 2] - positions[:-1, fid_r, 2]) ** 2
        feet_r = (((feet_r_x + feet_r_y + feet_r_z) < velfactor)).astype(np.float32)
        return feet_r, feet_l   
    feet_l, feet_r = foot_detect(positions, feet_thre)

    '''Quaternion and Cartesian representation'''
    r_rot = None

    def get_rifke(positions):
        '''Local pose'''
        positions[..., 0] -= positions[:, 0:1, 0]
        positions[..., 2] -= positions[:, 0:1, 2]
        '''All pose face Z+'''
        positions = qrot_np(np.repeat(r_rot[:, None], positions.shape[1], axis=1), positions)
        return positions
            
    def get_cont6d_params(positions):
        skel = Skeleton(n_raw_offsets, animo_kinematic_chain, "cpu")
        # (seq_len, joints_num, 4)
        quat_params = skel.inverse_kinematics_np(positions, face_joint_indx, smooth_forward=True)

        '''Quaternion to continuous 6D'''
        cont_6d_params = quaternion_to_cont6d_np(quat_params)
        # (seq_len, 4)
        r_rot = quat_params[:, 0].copy()
        #     print(r_rot[0])
        '''Root Linear Velocity'''
        # (seq_len - 1, 3)
        velocity = (positions[1:, 0] - positions[:-1, 0]).copy()
        #     print(r_rot.shape, velocity.shape)
        velocity = qrot_np(r_rot[1:], velocity)
        '''Root Angular Velocity'''
        # (seq_len - 1, 4)
        r_velocity = qmul_np(r_rot[1:], qinv_np(r_rot[:-1]))
        # (seq_len, joints_num, 4)
        return cont_6d_params, r_velocity, velocity, r_rot

    cont_6d_params, r_velocity, velocity, r_rot = get_cont6d_params(positions)
    positions = get_rifke(positions)

    '''Root height'''
    root_y = positions[:, 0, 1:2]

    '''Root rotation and linear velocity'''
    # (seq_len-1, 1) rotation velocity along y-axis
    # (seq_len-1, 2) linear velovity on xz plane
    r_velocity = np.arcsin(r_velocity[:, 2:3])
    l_velocity = velocity[:, [0, 2]]
    #     print(r_velocity.shape, l_velocity.shape, root_y.shape)
    root_data = np.concatenate([r_velocity, l_velocity, root_y[:-1]], axis=-1)  

    '''Get Joint Rotation Representation'''
    rot_data = cont_6d_params[:, 1:].reshape(len(cont_6d_params), -1)

    '''Get Joint Rotation Invariant Position Represention'''
    ric_data = positions[:, 1:].reshape(len(positions), -1)

    '''Get Joint Velocity Representation'''
    local_vel = qrot_np(np.repeat(r_rot[:-1, None], global_positions.shape[1], axis=1),
                        global_positions[1:] - global_positions[:-1])
    local_vel = local_vel.reshape(len(local_vel), -1)

    data = root_data
    data = np.concatenate([data, ric_data[:-1]], axis=-1)
    data = np.concatenate([data, rot_data[:-1]], axis=-1)
    data = np.concatenate([data, local_vel], axis=-1)
    data = np.concatenate([data, feet_l, feet_r], axis=-1)

    return data, global_positions, positions, l_velocity


In [ ]:
def recover_root_rot_pos(data):
    rot_vel = data[..., 0]
    r_rot_ang = torch.zeros_like(rot_vel).to(data.device)
    '''Get Y-axis rotation from rotation velocity'''
    r_rot_ang[..., 1:] = rot_vel[..., :-1]
    r_rot_ang = torch.cumsum(r_rot_ang, dim=-1)

    r_rot_quat = torch.zeros(data.shape[:-1] + (4,)).to(data.device)
    r_rot_quat[..., 0] = torch.cos(r_rot_ang)
    r_rot_quat[..., 2] = torch.sin(r_rot_ang)

    r_pos = torch.zeros(data.shape[:-1] + (3,)).to(data.device)
    r_pos[..., 1:, [0, 2]] = data[..., :-1, 1:3]
    '''Add Y-axis rotation to root position'''
    r_pos = qrot(qinv(r_rot_quat), r_pos)

    r_pos = torch.cumsum(r_pos, dim=-2)

    r_pos[..., 1] = data[..., 3]
    return r_rot_quat, r_pos

def recover_from_ric(data, joints_num):
    r_rot_quat, r_pos = recover_root_rot_pos(data)
    positions = data[..., 4:(joints_num - 1) * 3 + 4]
    positions = positions.view(positions.shape[:-1] + (-1, 3))

    '''Add Y-axis rotation to local joints'''
    positions = qrot(qinv(r_rot_quat[..., None, :]).expand(positions.shape[:-1] + (4,)), positions)

    '''Add root XZ to joints'''
    positions[..., 0] += r_pos[..., 0:1]
    positions[..., 2] += r_pos[..., 2:3]

    '''Concate root and joints'''
    positions = torch.cat([r_pos.unsqueeze(-2), positions], dim=-2)

    return positions
    
def calculate_distance(point1, point2):
    x1, y1, z1 = point1
    x2, y2, z2 = point2
    distance = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2 + (z2 - z1) ** 2)
    return distance

In [ ]:
with open('../AniMo4D/train.txt', 'r') as f:
    data_ls = f.readlines()
with open('../AniMo4D/test.txt', 'r') as f:    
    data_ls += f.readlines()
with open('../AniMo4D/val.txt', 'r') as f:
    data_ls += f.readlines()

data_ls = [x.strip() for x in data_ls]

In [ ]:
pattern = re.compile(r'_([a-z]+(?:_[a-z]+)?)_(male|female|juvenile)_(.*?)_keypoints')
categories = {}
for file_name in tqdm(data_ls):
    item=file_name.split('.')[1]
    match = pattern.search(item)
    if match:
        species = match.group(1).replace('_', ' ')
        gender = match.group(2)
        a_type=species+'+'+gender
        if a_type not in categories:
            categories[a_type] = []
        categories[a_type].append(file_name)

In [ ]:
l_idx1, l_idx2 = 23, 24
fid_r, fid_l = [19, 29], [15, 24]
face_joint_indx = [25,20,8,7]
r_hip, l_hip = 25,20
joints_num = 30
data_dir = './data_processed/'
save_dir1 = f'../AniMo4D/new_joints/'
save_dir2 =f'../AniMo4D/new_joint_vecs/'
os.makedirs(save_dir1, exist_ok=True)
os.makedirs(save_dir2, exist_ok=True)
with open('skel_template.pkl', 'rb') as file:
    skel_template = pickle.load(file)

In [ ]:
for category, files in tqdm(categories.items()):
    for index,filename in enumerate(files):
        source_data = np.load(os.path.join(data_dir, filename+'.npy'))[:, :joints_num]
        source_data[:, [9,10,11,12,13,14,15,16,17,18,19,20, 21, 22, 23, 24,25,26,27,28,29], :] = \
        source_data[:, [10,9,11,12,13,15,14,16,17,19,18,20, 24, 21, 23, 22,25,29,26,28,27], :]
        if index==0:
            n_raw_offsets = torch.from_numpy(animo_raw_offsets)
            tgt_skel = Skeleton(n_raw_offsets, animo_kinematic_chain, 'cpu')
            tgt_offsets = tgt_skel.get_offsets_joints(skel_template[category])
            bone_leng=calculate_distance(source_data[0,0,:],source_data[0,1,:])
        
        source_data=np.array(source_data)
        data, ground_positions, positions, l_velocity = process_file(source_data, bone_leng/200)
        rec_ric_data = recover_from_ric(torch.from_numpy(data).unsqueeze(0).float(), joints_num)
        joints_data_plot=rec_ric_data.squeeze().numpy()
        np.save(pjoin(save_dir1, filename), joints_data_plot)
        np.save(pjoin(save_dir2, filename), data)
            